In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%tensorflow_version 1.x
import tensorflow as tf 
tf.__version__
import numpy as np
import pandas as pd
import os
import string
from string import digits
import matplotlib.pyplot as plt
# %matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

TensorFlow 1.x selected.


In [5]:
# LOADING DATA 
data_path = "drive/MyDrive/Dataset/spoc-train.tsv"
def read_data(data_path):
    data = pd.read_csv(data_path, sep="\t")
    # data.head()
    return data 

data = read_data(data_path)
data.shape


(293854, 7)

In [6]:
# PREPROCESSING
def preprocess_data(data, sampleCount):
    # dropping unnecessary columns 
    data = data.drop(['workerid', 'probid', 'subid', 'line', 'indent'], axis=1)

    # dropping rows with NaN values 
    countNaN = pd.isnull(data).sum()
    print("Rows with NaN values: ", countNaN)
    df = data.dropna(axis=0, how="any")
    # df.head()
    lines = df
    print("Rows without NaN values: ", lines.shape)

    # picking random sample of length 
    lines=lines.sample(n=sampleCount,random_state=42)

    # lowercasing all characters
    lines['text']=lines['text'].apply(lambda x: x.lower())
    lines['code']=lines['code'].apply(lambda x: x.lower())

    # removing extra spaces
    lines['text']=lines['text'].apply(lambda x: x.strip())
    lines['code']=lines['code'].apply(lambda x: x.strip())
    
    # removing semicolon
    lines['text']=lines['text'].apply(lambda x: re.sub(";", " ", x))
    lines['code']=lines['code'].apply(lambda x: re.sub(";", " ", x))

    # removing brackets 
    lines['text']=lines['text'].apply(lambda x: re.sub("[\[({})\]]", " ", x))
    lines['code']=lines['code'].apply(lambda x: re.sub("[\[({})\]]", " ", x))

    lines['text']=lines['text'].apply(lambda x: re.sub(" +", " ", x))
    lines['code']=lines['code'].apply(lambda x: re.sub(" +", " ", x))

    # adding start and end tokens to target sequences
    lines['code'] = lines['code'].apply(lambda x : 'START_ '+ x + ' _END')


    # fetching all the distinct words in text and code columns 
    text_words = set()
    for text_word in lines['text']:
        for word in text_word.split():
            if word not in text_words:
                text_words.add(word)
        
    code_words = set()
    for code_word in lines['code']:
        for word in code_word.split():
            if word not in code_words:
                code_words.add(word)

    lines['text_length']=lines['text'].apply(lambda x:len(x.split(" ")))
    lines['code_length']=lines['code'].apply(lambda x:len(x.split(" ")))

    # lines[lines['text_length']<=20].shape, lines[lines['code_length']<=20].shape

    # clipping to 20 length
    lines=lines[lines['text_length']<=20]
    lines=lines[lines['code_length']<=20]

    print("maximum length of codes ",max(lines['code_length']))
    print("maximum length of texts ",max(lines['text_length']))

    max_length_src=max(lines['text_length'])
    max_length_tar=max(lines['code_length'])

    input_words = sorted(list(text_words))
    target_words = sorted(list(code_words))
    num_encoder_tokens = len(text_words)
    num_decoder_tokens = len(code_words)
    num_encoder_tokens, num_decoder_tokens

    num_decoder_tokens += 1 #for zero padding

    # indexing all the tokens for all the codes and texts 
    input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
    target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

    lines = shuffle(lines)
    lines.head(10)

    return lines, max_length_src, max_length_tar, input_token_index, target_token_index, num_encoder_tokens, num_decoder_tokens

lines, max_length_src, max_length_tar, input_token_index, target_token_index, num_encoder_tokens, num_decoder_tokens = preprocess_data(data, 75000)

Rows with NaN values:  text    77629
code        0
dtype: int64
Rows without NaN values:  (216225, 2)
maximum length of codes  20
maximum length of texts  20


In [7]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [8]:
# Splitting into training and test set  
X, y = lines['text'], lines['code']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
print("shape of train and test ", X_train.shape, X_test.shape)

shape of train and test  (58676,) (14670,)


In [9]:
def generate_batch(X = X_train, y = y_train, batch_size = 2):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            # one hot encoding 
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] 
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] 
                    if t>0:
                        # print("decoder_target_data[",i,t-1,target_token_index[word],"]")
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)        # yielding one hot encoded data


In [10]:
latent_dim=256

In [11]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)

encoder_lstm = LSTM(latent_dim, return_state=True)      
# return_state is set to True so that the last layer of encoder returns the output to the first layer of decoder  

encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]     # hidden state and cell state


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
# return_sequences and return_states are kept true so that the output of decoder states are captured 

decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [13]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    2407424     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    1963008     input_2[0][0]                    
______________________________________________________________________________________________

In [15]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 256
epochs = 50

In [16]:
history = model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs)


Epoch 1/50
229/229 [==============================] - 83s 362ms/step - loss: 1.1763
Epoch 2/50
229/229 [==============================] - 68s 298ms/step - loss: 0.7033
Epoch 3/50
229/229 [==============================] - 69s 301ms/step - loss: 0.5463
Epoch 4/50
229/229 [==============================] - 69s 303ms/step - loss: 0.4516
Epoch 5/50
229/229 [==============================] - 69s 301ms/step - loss: 0.3849
Epoch 6/50
229/229 [==============================] - 69s 300ms/step - loss: 0.3359
Epoch 7/50
229/229 [==============================] - 69s 301ms/step - loss: 0.2975
Epoch 8/50
229/229 [==============================] - 69s 300ms/step - loss: 0.2665
Epoch 9/50
229/229 [==============================] - 69s 302ms/step - loss: 0.2410
Epoch 10/50
229/229 [==============================] - 69s 302ms/step - loss: 0.2194
Epoch 11/50
229/229 [==============================] - 69s 302ms/step - loss: 0.2009
Epoch 12/50
229/229 [==============================] - 69s 303ms/step - lo

In [17]:
# Encode input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [18]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [19]:
model.save("model.h5")

In [20]:
! cp model.h5 drive/MyDrive/Dataset/latestmodel.h5

In [24]:
correct = 0
total = 10000

In [25]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [26]:
while k<10000:
    k+=1
    (input_seq, actual_output), _ = next(test_gen)
    decoded_sentence = decode_sequence(input_seq)
    if k<100:
      print('input pseudocode:', X_test[k:k+1].values[0])
      print('actual c code:', y_test[k:k+1].values[0][6:-4])
      print('predicted c code:', decoded_sentence[:-4])
      print()
    if (y_test[k:k+1].values[0][6:-4].replace(" ","") == decoded_sentence[:-4].replace(" ", "")):
      correct+=1

input pseudocode: n = integer
actual c code:  int n  
predicted c code:  int n 

input pseudocode: divide m by 10
actual c code:  m /= 10  
predicted c code:  m /= 10 

input pseudocode: create int arrays dis and sup of length 1200
actual c code:  int dis 1200 , sup 1200  
predicted c code:  int dis l n 

input pseudocode: declare int variables n and m
actual c code:  int n, m  
predicted c code:  int n, m 

input pseudocode: n,m,k=int
actual c code:  int n, m, k  
predicted c code:  

input pseudocode: for i = 0 to n exclusive
actual c code:  for int i = 0 i < n i++  
predicted c code:  for int i = 0 i < n i++ 

input pseudocode: set p i to -1
actual c code:  p i -= 2  
predicted c code:  p i = -1 

input pseudocode: create long long integer ans with ans = am / p
actual c code:  long long int ans = am / p  
predicted c code:  long long ans = / r 

input pseudocode: create integer variable c
actual c code:  int c  
predicted c code:  int c 

input pseudocode: read a then b then c
actua

In [27]:
print("Correct predictions: ", correct)

Correct predictions:  6021


In [28]:
print("Accuracy: ", correct/total)

Accuracy:  0.6021
